Goal: pick 4 USA men's Gymnasts. Gymnasts cannot compete on the same appartaus more than once. And each apparatus needs to be competed on 3 times each, then get the largest sum of the top 3 out of 4 scores on each apparats

good results:
- Variable Neighborhood Search (VNS)
- Hyper-Heuristics
- Tabu Search

Bad results:
- Harmony Search algorithm
- Simulated Annealing

In [43]:

import pandas as pd
import numpy as np
from itertools import combinations
from pyqubo import Array, Constraint, Placeholder, solve_qubo
from pulp import LpProblem, LpVariable, lpSum, LpMaximize



In [57]:
df = pd.read_csv('../../Data/cleandata22-23/usa_men22_23.csv')
df1 = pd.read_csv('../../Cleaning/USOPC/usa_men_apparatus.csv')
df2=pd.read_csv('update_usa_apparatus.csv')
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 

In [58]:
usa = iaa24[iaa24['Nation'] == 'USA']


In [59]:
df=usa.copy()

In [60]:
df1

,name,Rank,D_Score,E_Score,Penalty,Score,Apparatus
0,brody malone,2.200000,6.050000,8.353200,0.00,14.403200,hb
1,paul juda,3.800000,5.360000,8.666400,0.00,14.026400,hb
2,frederick nathaniel richard,24.166667,5.833333,7.761000,0.05,13.544333,hb
3,fred richard,11.333333,5.733333,7.783333,0.00,13.516667,hb
4,michael jaroh,6.000000,5.150000,8.350000,0.00,13.500000,hb
...,...,...,...,...,...,...,...
251,kameron nelson,32.000000,3.600000,6.700000,0.00,10.300000,ph
252,tyler flores,47.000000,4.400000,5.625000,0.00,10.025000,ph
253,kazuki hayashi,48.000000,3.850000,5.975000,0.00,9.825000,ph
254,drake andrews,47.500000,3.400000,7.925000,2.00,9.325000,ph


#  Variable Neighborhood Search (VNS):

Selected Athletes:
['asher hong' 'alex diab' 'ian gunther' 'yul moldauer']
Total Score:
339.653

---
['colt walker' 'paul juda' 'frederick richard' 'brody malone']
Total Score:
171.37124523809524


In [65]:
usa = iaa24[iaa24['Nation'] == 'USA']
df=usa.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

In [66]:
df

,Athlete,FX,PH,SR,VT,PB,HB
421,vitaliy guimaraes,14.250,14.350,13.600,14.450,13.900,13.40
422,khoi young,13.550,15.342,12.550,15.244,13.750,13.10
423,asher hong,14.029,12.949,14.566,16.680,14.355,10.45
424,yul moldauer,13.400,13.789,14.116,13.400,15.543,11.40
425,colt walker,12.450,12.750,13.250,14.897,15.055,13.10
...,...,...,...,...,...,...,...
716,ian skirkey,0.000,14.972,0.000,0.000,0.000,0.00
717,ignacio yockers,0.000,14.942,0.000,0.000,0.000,0.00
718,brandon dang,0.000,14.466,0.000,0.000,0.000,0.00
719,vahe petrosyan,0.000,0.000,0.000,13.350,0.000,0.00


In [67]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Define the neighborhood size
neighborhood_size = 2  # Adjust as needed

# Initialize the solution randomly
selected_athletes = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)

# Define a function to calculate the total score for a given solution
def calculate_total_score(selected_athletes):
    selected_scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].values
    top_3_scores = np.sum(np.partition(selected_scores, -3)[:, -3:])
    return top_3_scores

# Define the main VNS algorithm
def variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations):
    best_solution = None
    best_score = float('-inf')

    for iteration in range(num_iterations):
        # Shake: Generate a new solution by randomly changing athletes
        current_solution = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)
        
        for _ in range(neighborhood_size):
            # Local Search: Optimize the solution locally
            current_score = calculate_total_score(current_solution)
            
            # Perturb: Randomly change some athletes in the current solution
            perturbed_solution = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)
            perturbed_score = calculate_total_score(perturbed_solution)
            
            # Acceptance Criterion: Accept the perturbed solution if it improves the score
            if perturbed_score > current_score:
                current_solution = perturbed_solution
                current_score = perturbed_score

        # Update the best solution if the current one is better
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score

    return best_solution, best_score

# Run the VNS algorithm
best_solution, best_score = variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations=1000)

# Print the result
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)


Selected Athletes:
['fred richard' 'asher hong' 'riley loos' 'yul moldauer']
Total Score:
342.955


In [11]:
df = df2.copy()

In [12]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['fx', 'ph', 'sr', 'vt', 'pb', 'hb']

# Define the neighborhood size
neighborhood_size = 2  # Adjust as needed

# Initialize the solution randomly
selected_athletes = np.random.choice(df['name'], num_athletes_to_select, replace=False)

# Define a function to calculate the total score for a given solution
def calculate_total_score(selected_athletes):
    selected_scores = df[df['name'].isin(selected_athletes)][apparatuses].values
    top_3_scores = np.sum(np.partition(selected_scores, -3)[:, -3:])
    return top_3_scores

# Define the main VNS algorithm
def variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations):
    best_solution = None
    best_score = float('-inf')

    for iteration in range(num_iterations):
        # Shake: Generate a new solution by randomly changing athletes
        current_solution = np.random.choice(df['name'], num_athletes_to_select, replace=False)
        
        for _ in range(neighborhood_size):
            # Local Search: Optimize the solution locally
            current_score = calculate_total_score(current_solution)
            
            # Perturb: Randomly change some athletes in the current solution
            perturbed_solution = np.random.choice(df['name'], num_athletes_to_select, replace=False)
            perturbed_score = calculate_total_score(perturbed_solution)
            
            # Acceptance Criterion: Accept the perturbed solution if it improves the score
            if perturbed_score > current_score:
                current_solution = perturbed_solution
                current_score = perturbed_score

        # Update the best solution if the current one is better
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score

    return best_solution, best_score

# Run the VNS algorithm
best_solution, best_score = variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations=1000)

# Print the result
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)


Selected Athletes:
['colt walker' 'paul juda' 'khoi young' 'frederick nathaniel richard']
Total Score:
170.307825


# Hyper-Heuristics:

Best Selection: ['yul moldauer' 'mike fletcher' 'khoi young' 'fred richard']
Best Score: 253.48000000000002

---
Best Selection: ['colt walker' 'asher hong' 'brody malone' 'paul juda']
Best Score: 250.09701031746033


In [68]:
usa = iaa24[iaa24['Nation'] == 'USA']
df=usa.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])
df.head()

,Athlete,FX,PH,SR,VT,PB,HB
421,vitaliy guimaraes,14.250,14.350,13.600,14.450,13.900,13.40
422,khoi young,13.550,15.342,12.550,15.244,13.750,13.10
423,asher hong,14.029,12.949,14.566,16.680,14.355,10.45
424,yul moldauer,13.400,13.789,14.116,13.400,15.543,11.40
425,colt walker,12.450,12.750,13.250,14.897,15.055,13.10


In [69]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame
# df = ...

# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['fred richard' 'yul moldauer' 'colin van' 'vitaliy guimaraes']
Best Score: 253.89400000000003


In [18]:
df = df2.copy()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    58 non-null     object 
 1   fx      58 non-null     float64
 2   hb      58 non-null     float64
 3   pb      58 non-null     float64
 4   ph      58 non-null     float64
 5   sr      58 non-null     float64
 6   vt      58 non-null     float64
dtypes: float64(6), object(1)
memory usage: 3.3+ KB


In [20]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame
# df = ...

# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['fx', 'ph', 'sr', 'vt', 'pb', 'hb']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['name'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['name'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['khoi young' 'paul juda' 'brody malone' 'fuzzy benas']
Best Score: 250.58868690476191


# Tabu Search

['khoi young' 'yul moldauer' 'asher hong' 'fred richard']
Total Score:
516.4970000000001


---

['frederick nathaniel richard' 'khoi young' 'brody malone' 'paul juda']
Total Score:
253.19598690476192

In [70]:
usa = iaa24[iaa24['Nation'] == 'USA']
df=usa.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])
df.head()

,Athlete,FX,PH,SR,VT,PB,HB
421,vitaliy guimaraes,14.250,14.350,13.600,14.450,13.900,13.40
422,khoi young,13.550,15.342,12.550,15.244,13.750,13.10
423,asher hong,14.029,12.949,14.566,16.680,14.355,10.45
424,yul moldauer,13.400,13.789,14.116,13.400,15.543,11.40
425,colt walker,12.450,12.750,13.250,14.897,15.055,13.10


In [72]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score

# Assuming df is your DataFrame
# df = ...

# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 259.031
Iteration 2, Best Score: 262.457
Iteration 3, Best Score: 263.598
Iteration 4, Best Score: 263.598
Iteration 5, Best Score: 263.598
Iteration 6, Best Score: 263.598
Iteration 7, Best Score: 263.598
Iteration 8, Best Score: 263.598
Iteration 9, Best Score: 263.598
Iteration 10, Best Score: 263.598
Iteration 11, Best Score: 263.598
Iteration 12, Best Score: 263.598
Iteration 13, Best Score: 263.598
Iteration 14, Best Score: 263.598
Iteration 15, Best Score: 263.598
Iteration 16, Best Score: 263.598
Iteration 17, Best Score: 263.598
Iteration 18, Best Score: 263.598
Iteration 19, Best Score: 263.598
Iteration 20, Best Score: 263.598
Iteration 21, Best Score: 263.598
Iteration 22, Best Score: 263.598
Iteration 23, Best Score: 263.598
Iteration 24, Best Score: 263.598
Iteration 25, Best Score: 263.598
Iteration 26, Best Score: 263.598
Iteration 27, Best Score: 263.598
Iteration 28, Best Score: 263.598
Iteration 29, Best Score: 263.598
Iteration 30, Best Scor

In [27]:
df=df2.copy()

In [28]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['name'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop([ 'name'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['name'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['name']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score


# Set parameters
num_athletes_to_select = 4
tabu_size = 300
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 246.02389246031746
Iteration 2, Best Score: 251.2650757936508
Iteration 3, Best Score: 252.58625079365078
Iteration 4, Best Score: 253.19598690476192
Iteration 5, Best Score: 253.19598690476192
Iteration 6, Best Score: 253.19598690476192
Iteration 7, Best Score: 253.19598690476192
Iteration 8, Best Score: 253.19598690476192
Iteration 9, Best Score: 253.19598690476192
Iteration 10, Best Score: 253.19598690476192
Iteration 11, Best Score: 253.19598690476192
Iteration 12, Best Score: 253.19598690476192
Iteration 13, Best Score: 253.19598690476192
Iteration 14, Best Score: 253.19598690476192
Iteration 15, Best Score: 253.19598690476192
Iteration 16, Best Score: 253.19598690476192
Iteration 17, Best Score: 253.19598690476192
Iteration 18, Best Score: 253.19598690476192
Iteration 19, Best Score: 253.19598690476192
Iteration 20, Best Score: 253.19598690476192
Iteration 21, Best Score: 253.19598690476192
Iteration 22, Best Score: 253.19598690476192
Iteration 23, Best S

# CBC (Coin-or branch and cut) MILP solver 


In [78]:
usa = iaa24[iaa24['Nation'] == 'USA']
df=usa.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])
df.head()

,Athlete,FX,PH,SR,VT,PB,HB
421,vitaliy guimaraes,14.250,14.350,13.600,14.450,13.900,13.40
422,khoi young,13.550,15.342,12.550,15.244,13.750,13.10
423,asher hong,14.029,12.949,14.566,16.680,14.355,10.45
424,yul moldauer,13.400,13.789,14.116,13.400,15.543,11.40
425,colt walker,12.450,12.750,13.250,14.897,15.055,13.10


In [126]:
df=df2.copy()

In [127]:
df.head()

,name,fx,hb,pb,ph,sr,vt
0,aidan cuy,0.000000,0.000000,12.950000,0.0000,12.300000,13.800000
1,alex diab,13.225000,0.000000,0.000000,0.0000,14.144167,0.000000
2,asher hong,13.845625,12.072875,14.579444,13.1240,13.999778,14.393643
3,blake sun,0.000000,0.000000,13.673200,13.2232,0.000000,0.000000
4,brandon dang,0.000000,0.000000,0.000000,13.5500,0.000000,0.000000


In [125]:
# import pandas as pd
# from pulp import LpProblem, LpVariable, lpSum, value, LpMaximize



# # Create a binary variable for each gymnast and each apparatus
# gymnasts = df['name'].tolist()
# apparatus = ['fx', 'hb', 'pb', 'ph', 'sr', 'vt']

# x = LpVariable.dicts('x', [(g, a) for g in gymnasts for a in apparatus], 0, 1, LpVariable)

# # Create the optimization problem
# prob = LpProblem("GymnasticsOptimization", LpMaximize)

# # Objective function: maximize the sum of the top 3 scores on each apparatus
# prob += lpSum(df.loc[g, a] * x[(g, a)] for g in gymnasts for a in apparatus)

# # Constraint: Each apparatus is competed on 3 times
# for a in apparatus:
#     prob += lpSum(x[(g, a)] for g in gymnasts) == 3

# # Constraint: Each gymnast competes on each apparatus at most once
# for g in gymnasts:
#     prob += lpSum(x[(g, a)] for a in apparatus) <= 1

# # Solve the problem using the default solver
# prob.solve()

# # Display the results
# print("Status:", prob.status)

# # Extract selected gymnasts and their scores
# selected_gymnasts = [(g, a) for g in gymnasts for a in apparatus if value(x[(g, a)]) == 1]
# selected_scores = [df.loc[g, a] for g, a in selected_gymnasts]

# # Display selected gymnasts and their scores
# print("\nSelected Gymnasts:")
# for g, a in selected_gymnasts:
#     print(f"{g} on {a} - Score: {df.loc[g, a]}")

# # Display the largest sum of the top 3 scores on each apparatus
# print("\nLargest Sum of Top 3 Scores on Each Apparatus:", sum(sorted(selected_scores, reverse=True)[:3]))


# brute-force search

# Apparatus
Selected Athletes:
['aidan cuy', 'asher hong', 'paul juda', 'yul moldauer']
Maximum Score:
169.7672115079365

In [130]:
df=df2.copy()

In [137]:
df.head()

,name,fx,hb,pb,ph,sr,vt
0,aidan cuy,0.000000,0.000000,12.950000,0.0000,12.300000,13.800000
1,alex diab,13.225000,0.000000,0.000000,0.0000,14.144167,0.000000
2,asher hong,13.845625,12.072875,14.579444,13.1240,13.999778,14.393643
3,blake sun,0.000000,0.000000,13.673200,13.2232,0.000000,0.000000
4,brandon dang,0.000000,0.000000,0.000000,13.5500,0.000000,0.000000


Over 3 hours to run
100%|█████████████████████████████████| 424270/424270 [3:13:04<00:00, 36.62it/s]

Selected Athletes:
['aidan cuy', 'asher hong', 'paul juda', 'yul moldauer']
Maximum Score:
169.7672115079365


In [139]:
import pandas as pd
import itertools
from tqdm import tqdm

num_athletes = 4
apparatuses = ['fx', 'ph', 'sr', 'vt', 'pb', 'hb']

# Generate all possible combinations of athletes
athlete_combinations = list(itertools.combinations(df['name'], num_athletes))

# store the maximum score and the selected athletes
max_score = 0
selected_athletes = []

# Create a tqdm progress bar
progress_bar = tqdm(total=len(athlete_combinations))

# Iterate combination of athletes
for combination in athlete_combinations:
    # Generate all possible combinations of apparatuses for current athlete combination
    apparatus_combinations = list(itertools.combinations(apparatuses, num_athletes))

    # Iterate through each combination of apparatuses
    for apparatus_combination in apparatus_combinations:
        # Calculate sum of  top three scores for  current apparatus combination
        score = df[df['name'].isin(combination)][list(apparatus_combination)].sum(axis=1).nlargest(3).sum()

        # Update maximum score and selected athletes if a higher score is found
        if score > max_score:
            max_score = score
            selected_athletes = list(combination)

    # Update progress bar
    progress_bar.update(1)

# Close progress bar
progress_bar.close()

print('Selected Athletes:')
print(selected_athletes)
print('Maximum Score:')
print(max_score)

100%|█████████████████████████████████| 424270/424270 [3:13:04<00:00, 36.62it/s]

Selected Athletes:
['aidan cuy', 'asher hong', 'paul juda', 'yul moldauer']
Maximum Score:
169.7672115079365


In [140]:
usa = iaa24[iaa24['Nation'] == 'USA']
df=usa.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])
df.head()

,Athlete,FX,PH,SR,VT,PB,HB
421,vitaliy guimaraes,14.250,14.350,13.600,14.450,13.900,13.40
422,khoi young,13.550,15.342,12.550,15.244,13.750,13.10
423,asher hong,14.029,12.949,14.566,16.680,14.355,10.45
424,yul moldauer,13.400,13.789,14.116,13.400,15.543,11.40
425,colt walker,12.450,12.750,13.250,14.897,15.055,13.10


12 hours to run

100%|██████████████████████████████| 1581580/1581580 [11:59:24<00:00, 36.64it/s]

Selected Athletes:
['vitaliy guimaraes', 'khoi young', 'asher hong', 'yul moldauer']
Maximum Score:
173.974


In [141]:
import pandas as pd
import itertools
from tqdm import tqdm

num_athletes = 4
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Generate combinations of athletes
athlete_combinations = list(itertools.combinations(df['Athlete'], num_athletes))

# Initialize variables to store  maximum score and  selected athletes
max_score = 0
selected_athletes = []

progress_bar = tqdm(total=len(athlete_combinations))

# Iterate through  combination of athletes
for combination in athlete_combinations:
    # Generate combinations of apparatuses for  current athlete combination
    apparatus_combinations = list(itertools.combinations(apparatuses, num_athletes))

    # Iterate through combination of apparatuses
    for apparatus_combination in apparatus_combinations:
        # Calculate sum of top three scores for current apparatus combination
        score = df[df['Athlete'].isin(combination)][list(apparatus_combination)].sum(axis=1).nlargest(3).sum()

        # Update maximum score and selected athletes if a higher score is found
        if score > max_score:
            max_score = score
            selected_athletes = list(combination)

    progress_bar.update(1)

progress_bar.close()

print('Selected Athletes:')
print(selected_athletes)
print('Maximum Score:')
print(max_score)

100%|██████████████████████████████| 1581580/1581580 [11:59:24<00:00, 36.64it/s]

Selected Athletes:
['vitaliy guimaraes', 'khoi young', 'asher hong', 'yul moldauer']
Maximum Score:
173.974


# Simulated Annealing
- Simulated annealing involves randomness in accepting worse solutions to explore the search space effectively. After adding Seed the results are terrible


Selected Athletes: ['blake sun', 'khoi young', 'toby liang', 'blake sun']
Maximum Score: 163.632


In [33]:
usa = iaa24[iaa24['Nation'] == 'USA']
df=usa.copy()
df.head(1)

,Rank,Athlete,FX,PH,SR,VT,PB,HB,AA,round,Nation,year
421,1.0,vitaliy guimaraes,14.25,14.35,13.6,14.45,13.9,13.4,83.95,final,USA,2022


In [36]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    total_score = df[df['Athlete'].isin(selected_athletes)].iloc[:, 2:8].values
    total_score.sort(axis=1, kind='quicksort')
    return total_score[:, -3:].sum()

def simulated_annealing(df, num_athletes=4, iterations=1000, initial_temperature=1.0, cooling_rate=0.95):
    np.random.seed(88)  # Set the seed for reproducibility

    current_solution = np.random.choice(df['Athlete'], size=num_athletes, replace=False).tolist()
    current_score = calculate_score(current_solution, df)

    best_solution = current_solution.copy()
    best_score = current_score

    temperature = initial_temperature

    for iteration in range(iterations):
        neighbor_solution = current_solution.copy()
        idx1, idx2 = np.random.choice(num_athletes, size=2, replace=False)
        neighbor_solution[idx1], neighbor_solution[idx2] = neighbor_solution[idx2], neighbor_solution[idx1]

        neighbor_score = calculate_score(neighbor_solution, df)

        if neighbor_score > current_score or np.random.rand() < np.exp((neighbor_score - current_score) / temperature):
            current_solution = neighbor_solution
            current_score = neighbor_score

        if current_score > best_score:
            best_solution = current_solution.copy()
            best_score = current_score

        temperature *= cooling_rate

    return best_solution, best_score


# Run simulated annealing
best_athletes, best_score = simulated_annealing(df)

# Print the results
print('Selected Athletes:', best_athletes)
print('Maximum Score:', best_score)


Selected Athletes: ['colin van', 'ian gunther', 'brandon dang', 'asher hong']
Maximum Score: 226.752


# Harmony Search algorithm = results for 6 and not 4 athletes

              Athlete     FX      PH     SR     VT      PB     HB
715    patrick hoopes   0.00  14.992   0.00   0.00   0.000   0.00
708   curran phillips   0.00   0.000   0.00   0.00  15.997  13.70
427   ian lasic-ellis  13.65  13.500  13.75  13.55  13.300  13.30
711   spencer goodell  13.45   0.000   0.00   0.00  13.750   0.00
442      noah newfeld  12.20  13.050  12.10  13.50  13.200  12.15
444  garrett braunton  12.85  12.900  13.05  12.20  13.150  11.65

In [37]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv')
usa = iaa24[iaa24['Nation'] == 'USA']
df=usa.copy()

In [38]:


import pandas as pd
import numpy as np

def fitness(solution, df):
    selected_athletes = df.iloc[solution]
    
    # Check if each apparatus is competed exactly three times
    apparatus_counts = selected_athletes.groupby('Athlete').size()
    if not all(count == 3 for count in apparatus_counts):
        return np.inf
    
    # Check if the selected athletes are unique for each apparatus
    if len(set(solution)) < len(solution):
        return np.inf
    
    scores = selected_athletes[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].values
    top3_sum = np.sum(np.partition(scores, -3)[:, -3:], axis=1)
    
    return -np.sum(top3_sum)  # Negative because Harmony Search seeks to minimize the objective function

def harmony_search(df, num_iterations, harmony_memory_size):
    num_athletes = len(df)
    num_apparatus = 6  # Assuming 6 apparatus (FX, PH, SR, VT, PB, HB)
    hm_size = harmony_memory_size

    # Initialize harmony memory with random solutions
    harmony_memory = np.random.randint(0, num_athletes, size=(hm_size, num_apparatus))

    for iteration in range(num_iterations):
        # Create new harmony solutions
        new_solution = np.random.randint(0, num_athletes, size=num_apparatus)

        # Update the harmony memory with the new solution
        worst_index = np.argmin([fitness(harmony, df) for harmony in harmony_memory])
        
        # Check if the new solution is valid before updating the memory
        if fitness(new_solution, df) < np.inf:
            harmony_memory[worst_index] = new_solution

    # Find the best solution in the harmony memory
    best_solution = harmony_memory[np.argmin([fitness(harmony, df) for harmony in harmony_memory])]

    return df.iloc[best_solution]

# Assuming 'your_dataframe.csv' is the path to your CSV file
# df = pd.read_csv('your_dataframe.csv')

# Set the number of iterations and harmony memory size
num_iterations = 100
harmony_memory_size = 10

# Run the Harmony Search algorithm
selected_athletes = harmony_search(df, num_iterations, harmony_memory_size)

# Print the selected athletes
print(selected_athletes[['Athlete', 'FX', 'PH', 'SR', 'VT', 'PB', 'HB']])


             Athlete     FX      PH     SR     VT     PB     HB
689     landen blixt  13.50  12.800  12.45  12.65  13.80  13.20
440     jack freeman  13.25  11.200  12.80  13.95  12.05  13.35
450  ignacio yockers  11.75  12.893  11.45  13.60  11.95  11.40
705   michael artlip   0.00  11.000  11.65   0.00  11.30  12.65
715   patrick hoopes   0.00  14.992   0.00   0.00   0.00   0.00
451   kazuki hayashi  12.20   9.600  12.25  14.60  12.75  11.25
